In [1]:
!nvidia-smi

Thu Dec 25 11:21:34 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [7]:
%%writefile vecadd.cu
#include <cuda_runtime.h>
#include <memory.h>
#include <cstdlib>
#include <ctime>
#include <stdio.h>
#include <cmath>
#include <iostream>
#include <chrono>

__global__ void vecADD(float* A, float* B, float* C, int vectorLength){
  int workIndex=threadIdx.x+blockIdx.x*blockDim.x;
  if (workIndex<vectorLength){
    C[workIndex]=A[workIndex]+B[workIndex];
  }
}

void initArray(float* A, int length){
  for(int i=0; i<length; i++){
    A[i]=std::rand()/(float)RAND_MAX;
  }
}

void serialVecAdd(float* A, float* B, float* C, int length){
for(int i=0; i<length; i++){
C[i] = A[i] + B[i];
}
}

bool vectorApproximatelyEqual(float* A, float* B, int length, float epsilon=1e-5f){
for(int i=0; i<length; i++){
if(fabs(A[i]-B[i]) > epsilon){
printf("Index %d mismatch: %f != %f", i, A[i], B[i]);
return false;
}
}
return true;
}



void explicitMem(int vectorLength, int threads){
  float* A, *B, *C; //pointers for host memory
  float* devA, *devB, *devC; //device memory
  cudaMallocHost(&A, vectorLength*sizeof(float));
  cudaMallocHost(&B, vectorLength*sizeof(float));
  cudaMallocHost(&C, vectorLength*sizeof(float));

  initArray(A, vectorLength);
  initArray(B, vectorLength);

  cudaMalloc(&devA, vectorLength*sizeof(float));
  cudaMalloc(&devB, vectorLength*sizeof(float));
  cudaMalloc(&devC, vectorLength*sizeof(float));

  cudaMemcpy(devA, A, vectorLength*sizeof(float), cudaMemcpyDefault);
  cudaMemcpy(devB, B, vectorLength*sizeof(float), cudaMemcpyDefault);
  cudaMemset(devC, 0, vectorLength*sizeof(float));

  int blocks = (vectorLength + threads-1)/threads;
  auto start = std::chrono::high_resolution_clock::now();
  vecADD<<<blocks, threads>>>(devA, devB, devC, vectorLength);
  cudaError_t err = cudaGetLastError();
if (err != cudaSuccess) {
    printf("Kernel launch failed: %s\n", cudaGetErrorString(err));
}

cudaDeviceSynchronize();
  cudaMemcpy(C, devC, vectorLength*sizeof(float), cudaMemcpyDefault);
  auto end = std::chrono::high_resolution_clock::now();
  double time_ms =std::chrono::duration<double, std::milli>(end - start).count();

 float* comparisonResult = new float[vectorLength]();
 serialVecAdd(A, B, comparisonResult, vectorLength);

  if(vectorApproximatelyEqual(C, comparisonResult, vectorLength)){
printf("Explicit Memory: CPU and GPU answers match\n");
}
else{
printf("Explicit Memory: Error- CPU and GPU answers to not match\n");
}

  std::cout << "Time: " << time_ms << " ms\n";
  std::cout <<"Total number of threads" <<blocks*threads <<"\n";
  cudaFreeHost(A);
  cudaFreeHost(B);
  cudaFreeHost(C);
  cudaFree(devA);
  cudaFree(devB);
  cudaFree(devC);
  delete[] comparisonResult;
}




int main(int argc, char** argv){
  int vectorLength=1000;
  std::srand(std::time(nullptr));   //
  if(argc>=2){
    vectorLength=std::atoi(argv[1]);
  }
  int threads[4]={32, 128, 256, 512};
  for (int i=0; i<4; i++){
    explicitMem(vectorLength, threads[i]);
  }
  return 0;
}


Overwriting vecadd.cu


In [8]:
!nvcc vecadd.cu \
  --generate-code arch=compute_75,code=sm_75 \
  -o vecadd

In [9]:
!./vecadd 1000


Explicit Memory: CPU and GPU answers match
Time: 0.147044 ms
Total number of threads1024
Explicit Memory: CPU and GPU answers match
Time: 0.031673 ms
Total number of threads1024
Explicit Memory: CPU and GPU answers match
Time: 0.031055 ms
Total number of threads1024
Explicit Memory: CPU and GPU answers match
Time: 0.030523 ms
Total number of threads1024


In [22]:
!./vecadd 100000

Explicit Memory: CPU and GPU answers match
Time: 0.1952 ms
Total number of threads100000
Explicit Memory: CPU and GPU answers match
Time: 0.06351 ms
Total number of threads100096
Explicit Memory: CPU and GPU answers match
Time: 0.065435 ms
Total number of threads100096
Explicit Memory: CPU and GPU answers match
Time: 0.066242 ms
Total number of threads100352


In [21]:
!./vecadd 10000000

Explicit Memory: CPU and GPU answers match
Time: 4.29066 ms
Total number of threads10000000
Explicit Memory: CPU and GPU answers match
Time: 3.73502 ms
Total number of threads10000000
Explicit Memory: CPU and GPU answers match
Time: 3.71921 ms
Total number of threads10000128
Explicit Memory: CPU and GPU answers match
Time: 3.70961 ms
Total number of threads10000384


In [20]:
import numpy as np
import time

a=np.random.rand(1000)
b=np.random.rand(1000)
start = time.perf_counter()
c=a+b
end = time.perf_counter()
time_ms = (end - start) * 1000
print(f"Time: {time_ms:.3f} ms")

Time: 0.112 ms
